# IMPORTING JSON FILE

In [ ]:
import json

with open('"C:\\Users\\Aditya\\Documents\\chAt_bot\\intents3.json"') as file:
    intents = json.load(file, strict = False) 
intents = intents['intents']

In [ ]:
print("[", end = "")
for intent in intents:
  print("{", end = "")
  for key, value in intent.items():
    print("{}: {},".format(key, value))
  print("\b\b\n},")
print("\b\b]")

# IMPORTING LIBRARIES

In [ ]:
import numpy as np
import tensorflow as tf

import tflearn
import random
import pickle

In [ ]:
import nltk
#nltk.download('all')

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

# PRE-PROCESSING THE DATA

In [ ]:
retrain_model = True

if retrain_model:
    #This will be a list of all the words used in any of the 'patterns' in each intent
    all_words = []
    #This will be a list of all the 'tag's associated with the intents
    all_tags = [] 
    #This will be a list containing all of the 'patterns' for each intent where each individual pattern is grouped together
    intent_patterns = []
    #This will be a list correlated with 'intent_patterns' where every pattern in 'intent_patterns' is correlated with its respective tag in this list
    intent_tags = [] 
    
    #Here we fill in all of the lists above. Note that we tokenize the words in each pattern which means we split each pattern into individual words
    for intent in intents:
        for pattern in intent['patterns']:
            words = nltk.word_tokenize(pattern)

            all_words.extend(words)
            intent_patterns.append(words)
            intent_tags.append(intent['tag'])
            
        all_tags.append(intent['tag'])
        
    #Here we stem the words in all_words. This means that we reduce every word down to its root form or stem. This will prevent our chatbot from confusin
    #Very similar words with eachother. For example, the chatbot might normally confuse the words 'running' and 'run' because they appear different even 
    #Though they effectively mean the same thing. Stemming will reduce both of these words down to their root form which would be 'run' so the chatbot will
    #No longer be confused  
    all_words = [stemmer.stem(word.lower()) for word in all_words]
    all_words = sorted(list(set(all_words)))
    
    all_tags = sorted(all_tags)
    
    x_train = []
    y_train = []
    
    y_empty = [0 for i in range(len(all_tags))]
    
   #Here we are creating our training set input and output values for our deep learning algorithm
    #We will do this by iterating through our intents and turning each one into a bag of words, or a vector that indicates which words are in each pattern.
    #These bags of words will be the x values and the y values will be the intent that each bag of words is associated with.
    #The machine learning will train on this data and will be able to determine which bag of words 
    for index, intent in enumerate(intent_patterns):
        bag_of_words = []
        
        intent_words = [stemmer.stem(word.lower()) for word in intent]
        
        for word in all_words:
            if word in intent_words:
                bag_of_words.append(1)
            else:
                bag_of_words.append(0)
                
        one_hot_encode_y = y_empty[:]
        one_hot_encode_y[all_tags.index(intent_tags[index])] = 1
        
        x_train.append(bag_of_words)
        y_train.append(one_hot_encode_y)
    
    #Here is the data we will be using to train our neural network later
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    #Here we just save our training data so we don't need to process it again if we just want to run our chatbot
    with open('training_data.pickle', 'wb') as f:
        pickle.dump((all_words, all_tags, x_train, y_train), f)
else:
    with open('training_data.pickle', 'rb') as f:
        all_words, all_tags, x_train, y_train = pickle.load(f)

# MODEL BUILDING

In [ ]:
#tf.reset_default_graph()

#Creation the neural network layers
neural_net = tflearn.input_data(shape = [None, len(x_train[0])])
neural_net = tflearn.fully_connected(neural_net, 8)
neural_net = tflearn.fully_connected(neural_net, 8)
#Here we use the softmax activation function so the output of our neural network is a probability. We will make use of this later
neural_net = tflearn.fully_connected(neural_net, len(y_train[0]), activation = 'softmax')
neural_net = tflearn.regression(neural_net)

model = tflearn.DNN(neural_net)

# BATCH GRADIENT DESCEND 

In [ ]:
if retrain_model:
    #Here we train the neural network with the training data we created in the NLP stage
    model.fit(x_train, y_train, n_epoch = 1000, batch_size = 8, show_metric = True)
    model.save('model.tfl')
else:
    model.load('./model.tfl')

In [ ]:
def text_to_bag(text, all_words):
    #Initialize the bag of words by creating an empty slot for every word in the vector
    bag_of_words = [0 for i in range(len(all_words))]
    
    #First we split up the input into individual words and stem them so they match the same format as in our vector
    text_words = nltk.word_tokenize(text)
    text_words = [stemmer.stem(word.lower()) for word in text_words]
    
    #Now we create the bag of words by filling in a 1 for the words that the user used
    for word in text_words:
        if word in all_words:
            bag_of_words[all_words.index(word)] = 1
    
    #And return the bag of words
    return np.array(bag_of_words)

# SPEECH RECOGNITION

In [ ]:
import speech_recognition as sr
r = sr.Recognizer()
import pyttsx3
engine = pyttsx3.init()

In [ ]:
def bot_speaking(message):
    engine.say(message)
    engine.runAndWait()
    if engine._inLoop:
        engine.endLoop()

In [ ]:
def get_input():
    with sr.Microphone() as source:
        #print("Say something!!!")
        bot_speaking("Hey mate say something!")
        audio = r.listen(source,timeout=0)
        bot_speaking("Perfect, Thanks!")
    try:
        msg = r.recognize_google(audio)
        print("TEXT: "+msg);
        bot_speaking("you said "+msg)
        return msg
    except:
        bot_speaking("Sorry mate! It's not working")
        pass;

# CHAT FUNCTION

In [ ]:
def chat():
    #Starting message
    print("BOT : Hi! I am your assistance bot. I am here to answer your queries")
    
    #Reset the context state since there is no context at the beginning of the conversation
    context_state = None
    
    #This is what the bot will say if it doesn't understand what the user is saying
    default_responses = ['Sorry, Im not sure I know what you mean! You could try rephrasing that or saying something else!',
                         'You confuse me human. Lets talk about something else.',
                         'Im not sure what that means and I dont really care. Lets talk about something else',
                         'I dont understand that! Try rephrasing or saying something else.']
    
    #This chat loop will go on forever until the user types quit
    while True:
        user_chat = str(input('YOU : '))
        if user_chat.lower() == 'quit':
            break
            
        #Convert chat to bag of words
        user_chat_bag = text_to_bag(user_chat, all_words)
        
        #Pass bag of words into our neural network
        response = model.predict([user_chat_bag])[0]
        
        #Get the intent that the bag of words is most highly correlated with
        response_index = np.argmax(response)
        response_tag = all_tags[response_index]
        
        #If the neural network is fairly certain that it has chosen the right intent (and isnt randomly guessing)
        #In this case, we will only get a response if the neural network is more than 80% certain
        if response[response_index] > 0.8:
            for intent in intents:
                #Get the intent that is predicted
                if intent['tag'] == response_tag:
                    #Check if this response is associated with a specific context
                    if 'context_filter' not in intent or 'context_filter' in intent and intent['context_filter'] == context_state:
                        #Get all of the possible responses from this intent
                        possible_responses = intent['responses']
                        #If this intent is associated with a context set, then set the context state
                        if 'context_set' in intent:
                            context_state = intent['context_set']
                        else:
                            context_state = None
                        #Select a random message from the intent responses
                        ms = random.choice(possible_responses)
                        print("BOT : "+ms)
                        bot_speaking(ms)
                    else:
                        ms = random.choice(default_responses)
                        print("BOT : "+ms)
                        bot_speaking(ms)
        else:
            ms = random.choice(default_responses)
            print("BOT : "+ms)
            bot_speaking(ms)
chat()